# Tuning LLM with Learning by Examples

Adapter le modele génératif de langage pour ajouter un module "auto-critic". Concretement, cela consiste à ajouter une sortie à derniere sequence de decodeur. Entrainer le modele en figeant les poids du LLM (TransfertLearning), puis fineTuner le modele complet.

![LLM-Critic](LLM-AutoCritic.png)

Nous nous appuyons sur un grand modèle de langage (LLM) open-source, qui peut être affiné et adapté selon les besoins. Le processus d'ajustement se déroule en deux-trois étapes : tout d'abord, nous ajustons les poids du réseau en fonction d'un problème spécifique, puis nous optimisons le "prompt" à fournir au réseau en amont afin de répondre au mieux à la demande. Enfin, il est possible de "performer" le modèle par des outils d'auto-apprentissage.


Dans cette première étape, nous aborderons un cas d'école pour illustrer le processus. Nous verrons ensuite comment construire le jeu de données pour atteindre notre objectif. Ce tutoriel est basé sur le modèle [Falcon-7B](https://huggingface.co/tiiuae/falcon-7b) et utilise le guide Colab suivant : https://colab.research.google.com/drive/1Vvju5kOyBsDr7RX_YAvp6ZsSOoSMjhKD. Pour l'entraînement et les inférences, nous utiliserons Colab, car il nous donne gratuitement accès à un GPU T4 (la durée d'accès gratuite est à définir) et à un prix avantageux pour des GPU plus puissants tels que l'A100 et le V100.

*Nous aborderons les points suivants : :*

    - Comment utiliser un modèle de langage pré-entraîné pour la génération de texte ?
    - Comment créer des données d'entraînement pour le fine-tuning ?
    - Comment lancer l'ajustement d'un modèle LLM pour une problématique d'instruction classique ?
    
 ### Bibliographie :
 
 
 - [LLaMA](https://arxiv.org/abs/2302.13971v1)
 - [Datasets](https://arxiv.org/abs/2109.02846)
 - [QLoRA](https://arxiv.org/abs/2305.14314)
 
 
 ### Prérequis : 


    - Python 3.8
    - Pytorch 2.0
    - 8Gb RAM

In [1]:
import torch

## Import Model

La première étape consiste à importer le modèle pré-entraîné, offrant deux options avec **PyTorch** : utiliser directement la bibliothèque en chargeant les paramètres pré-entraînés depuis les fichiers "bin" téléchargés, ou instancier le modèle LLM souhaité et charger les "checkpoints" au format "dict". Dans le premier cas, on crée une instance du modèle et on charge les poids à partir des fichiers "bin". Dans le second cas, on instancie le modèle LLM et on le charge avec les "checkpoints". Assurez-vous de spécifier correctement les chemins des fichiers requis. Remplacez "pytorch_model" par la classe correspondante au modèle LLM.

In [ ]:
### Par définition du modèle
model = MyModel()
# Chargement des checkpoints
checkpoint = torch.load("chemin/vers/le/fichier.pth")
model.load_state_dict(checkpoint['model_state_dict'])
### Par le binaire
model = torch.load('pytorch_model.bin', map_location='cpu')

Une autre possibilité est d'utiliser la bibliothèque clé en main "transformers" de **Hugging Face**. L'avantage de cette approche est qu'elle simplifie le processus en une seule étape et offre des fonctions prédéfinies pour le fine-tuning, ce qui facilite grandement son utilisation. Par conséquent, nous utiliserons exclusivement cet outil pour nos besoins de fine-tuning à l'avenir, car il offre une solution pratique et efficace.

In [2]:
import transformers

Pour importer un modèle pré-entraîné et visualiser sa structure de réseau, vous pouvez suivre cette étape.

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", trust_remote_code=True)
print(model)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

Le modèle LLaMA, bien qu'un des premiers modèles LLM "décodeur seulement" pour la complétion de texte disponibles en local, présente des limitations de licence. Par conséquent, nous opterons plutôt pour le modèle Falcon-7B, basé sur le modèle BLOOM, qui offre une alternative plus favorable. À partir de ce point, nous utiliserons exclusivement le modèle Falcon-7B pour nos besoins.

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear(in_features=4544, out_features=4672, bias=False)
          (dense): Linear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)


Ensuite, pour définir la structure des données d'entrée pour notre modèle, nous devons segmenter les mots en unités plus petites appelées "tokens". Pour ce faire, nous utilisons une méthode appelée "AutoTokenizer" qui convertit une phrase en un ensemble de "tokens" en utilisant un modèle spécifique. Chaque modèle dispose d'un "Tokenizer" spécifique, et son utilisation se fait de la manière suivante : 

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
input_ = tokenizer("Girafatron is nice.\nFabien: Hello, Girafatron!\nGirafatron:", return_tensors="pt")

Une fois que la séquence a été convertie pour être utilisée dans un modèle, nous pouvons utiliser la fonction "generate" pour compléter la séquence en générant du texte supplémentaire. Cette fonction permet au modèle de prédire les mots ou les phrases suivantes en fonction du contexte fourni.

In [7]:
output_ids = model.generate(input_.input_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 26, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Enfin, nous décodons et affichons le résultat de la génération en utilisant à nouveau le tokenizer. Cela nous permet de convertir les "tokens" générés par le modèle en texte lisible. En utilisant le tokenizer, nous pouvons restaurer la séquence de mots ou de phrases complétées pour l'afficher et l'examiner.

In [8]:
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Girafatron is nice.
Fabien: Hello, Girafatron!
Girafatron: Hello


Il est important de noter que la génération de texte peut être un processus très intensif, en particulier sur un ordinateur avec une quantité limitée de RAM. Pour remédier à cela, une approche consiste à "quantifier" le modèle en convertissant ses poids dans un format de données plus compact. Cette technique permet de réduire la consommation de mémoire et d'accélérer le processus de génération. Nous utiliserons cette méthode lors du fine-tuning du modèle pour optimiser ses performances.

## Prepare Fine Tuning

Le fine-tuning est une technique d'apprentissage par transfert qui consiste à prendre un modèle pré-entraîné et à y apporter des modifications. Il existe différentes stratégies de fine-tuning : certaines consistent à ajouter un nouveau modèle à entraîner entièrement à la suite du modèle pré-entraîné, tandis que d'autres impliquent une modification directe des poids du réseau pré-entraîné. Le choix de la stratégie dépend des besoins spécifiques de la tâche et des ressources disponibles.

Pour faciliter le processus de fine-tuning, il existe une bibliothèque tout-en-un de **Hugging Face** appelée "peft" (Parameter-Efficient Fine-Tuning). Cette bibliothèque offre des fonctionnalités spécifiquement conçues pour le fine-tuning des modèles génératif, permettant ainsi une implémentation simplifiée et efficace de cette étape cruciale. En utilisant "peft", vous pouvez bénéficier de diverses fonctionnalités telles que la gestion des données d'entraînement, l'optimisation des hyperparamètres et la création de boucles d'entraînement, ce qui facilite grandement le processus de fine-tuning.

In [9]:
import peft


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Python38\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary C:\Python38\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable


C:\Python38\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Pour convertir le modèle pour réduire la taille par quantification, nous faisons :

In [10]:
model.gradient_checkpointing_enable()
model_kbit = peft.prepare_model_for_kbit_training(model)

Dans le cadre du fine-tuning, nous allons modifier directement les poids du réseau, en nous concentrant uniquement sur ceux qui n'ont pas d'effet significatif sur la sortie ou qui ont le même effet. Pour cela, nous construisons une fonction spécifique qui nous permet de déterminer quels poids peuvent être modifiés dans notre modèle. Cette approche ciblée nous permet de concentrer l'entrainement sur les parties du modèle qui ont le plus d'impact sur la tâche spécifique que nous souhaitons accomplir, tout en préservant les parties du modèle qui sont déjà efficaces et bien entraînées.

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

Pour le fine-tuning, nous utilisons l'algorithme LoRa (Low Rank), qui nous permet de déterminer les poids modifiables dans notre modèle. L'algorithme LoRa identifie les poids qui ont un impact significatif sur la sortie du modèle et sélectionne les poids qui ont une influence moindre ou similaire. En réduisant le rang des poids, nous pouvons réduire la dimensionnalité du modèle d'entrainement, ce qui facilite le fine-tuning en réduisant la complexité et en améliorant l'efficacité du processus d'entraînement. Cette approche permet d'optimiser les performances du modèle tout en limitant le nombre de poids à ajuster, ce qui peut être particulièrement utile lorsque les ressources computationnelles sont limitées.

In [12]:
config = peft.LoraConfig(r=8, lora_alpha=32, target_modules=["query_key_value"],  lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

model_kbit = peft.get_peft_model(model_kbit, config)
print_trainable_parameters(model_kbit)

trainable params: 2359296 || all params: 6924080000 || trainable%: 0.03407378308742822


L'avantage est que seuls 0,03% des poids du modèle doivent être modifiés. Cette proportion faible réduit considérablement les exigences en termes de ressources de calcul. Nous pouvons donc effectuer le fine-tuning de manière efficace, même avec des ressources limitées, tout en préservant les performances du modèle pré-entraîné. Cela permet des ajustements précis et rapides pour répondre aux besoins spécifiques de la tâche sans nécessiter une grande puissance de calcul.

### Prepare Data

Pour entraîner notre modèle, nous avons besoin de données d'entraînement qui correspondent à la tâche spécifique que nous souhaitons accomplir. Pour cela, nous utilisons le module "datasets" qui offre un ensemble de jeux de données pré-construits. Cependant, il est également possible de construire des données spécifiques pour une application particulière si nécessaire. En utilisant le module "datasets", nous pouvons accéder à des ensembles de données diversifiés et bien organisés, ce qui facilite le processus d'entraînement et d'évaluation de notre modèle. Il est important de sélectionner ou de créer des données d'entraînement de haute qualité et représentatives de la tâche que nous souhaitons résoudre afin d'obtenir les meilleurs résultats possibles lors du fine-tuning.

In [13]:
import datasets

Dans notre cas d'école, nous allons utiliser des données de citations comprenant un titre, une citation correspondante et l'auteur. Notre objectif sera de prédire une nouvelle citation possible à partir d'un titre donné. Pour préparer ces données, nous devons les tokenizer, c'est-à-dire les convertir en une représentation numérique compréhensible par le modèle. Nous utilisons donc une fonction de tokenization spécifique pour diviser les titres et les citations en unités appelées "tokens". Cette étape de tokenization nous permet pour préparer les données d'entrée au modèle et de les traiter de manière adéquate lors du processus d'entraînement.

In [14]:
data = datasets.load_dataset("Abirate/english_quotes")
data_tokenized = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)
print(data.num_rows)

Found cached dataset json (C:/Users/ffurfaro/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\ffurfaro\.cache\huggingface\datasets\Abirate___json\Abirate--english_quotes-6e72855d06356857\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-b19fbca03c240ead.arrow


{'train': 2508}


Pour visualiser les "premières données", nous faisons :

In [31]:
print(data_tokenized.data['train'][0][:3], data_tokenized.data['train'][1][:3], data_tokenized.data['train'][2][:3])

[
  [
    "“Be yourself; everyone else is already taken.”",
    "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
    "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”"
  ]
] [
  [
    "Oscar Wilde",
    "Marilyn Monroe",
    "Albert Einstein"
  ]
] [
  [
    [
      "be-yourself",
      "gilbert-perreira",
      "honesty",
      "inspirational",
      "misattributed-oscar-wilde",
      "quote-investigator"
    ],
    [
      "best",
      "life",
      "love",
      "mistakes",
      "out-of-control",
      "truth",
      "worst"
    ],
    [
      "human-nature",
      "humor",
      "infinity",
      "philosophy",
      "science",
      "stupidity",
      "universe"
    ]
  ]
]


Bien que notre ensemble de données soit limité à seulement 2500 exemples, il est suffisant pour notre cas d'utilisation spécifique. Malgré sa taille réduite, cet ensemble de données couvre une gamme variée de titres et de citations, ce qui nous permettra de former un modèle capable de générer des citations pertinentes en fonction des titres fournis.

### Train

Lors de l'entraînement, nous utilisons les paramètres suivants pour optimiser notre modèle :

In [32]:
# needed for falcon
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model_kbit,
    train_dataset=data_tokenized["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        #fp16=True, # if cuda device
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Pour lancer l'entraînement de notre modèle en utilisant la bibliothèque Hugging Face, nous utilisons la fonction suivante :

In [ ]:
model_kbit.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Pour mieux comprendre cette étape, regardez la vidéo de [1littlecoder](https://youtu.be/NRVaRXDoI3g).

### Save Model et Import

Pour enregistrer les poids modifiés de notre modèle, nous utilisons la méthode fournie par la bibliothèque Hugging Face. Cela nous permet de sauvegarder les paramètres ajustés du réseau dans un format approprié, tel qu'un fichier binaire ou un dictionnaire, en fonction de nos besoins. L'enregistrement des poids modifiés nous permet de conserver les modifications apportées lors du processus de fine-tuning, afin de pouvoir les réutiliser ultérieurement pour des inférences ou pour continuer l'entraînement du modèle. Cette étape est essentielle pour préserver et partager les résultats de notre travail.

In [46]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

Une fois que nous avons enregistré les poids modifiés de notre modèle, nous pouvons facilement les importer en utilisant une fonction dédiée. Cette fonction nous permet de charger spécifiquement les poids que nous avons ajustés, en les extrayant du fichier ou du dictionnaire dans lequel ils ont été enregistrés.

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model_trained = get_peft_model(model_kbit, lora_config)

En important uniquement les poids modifiés, nous pouvons réutiliser notre modèle fine-tuné pour effectuer des inférences ou poursuivre l'entraînement sans avoir besoin de reconfigurer l'ensemble du réseau. Cela nous offre une flexibilité et une efficacité accrues dans l'utilisation de notre modèle pré-entrainé et adapté à notre tâche spécifique.

## Conclusion et Perspective

Il est intéressant de constater que même avec un ensemble de données d'entraînement relativement restreint, il est possible d'optimiser efficacement un modèle de langage LLM pré-entraîné pour une utilisation spécifique. Cette approche de fine-tuning permet d'adapter le modèle existant à notre tâche spécifique de manière pratique et efficiente. Grâce à cette méthode, il est possible de bénéficier des connaissances préalables du modèle tout en l'adaptant pour obtenir des résultats pertinents et précis dans notre domaine d'intérêt. Cela démontre la flexibilité des techniques de fine-tuning dans le domaine de l'apprentissage automatique des LLM et ouvre des possibilités intéressantes pour l'optimisation de modèles pré-entraînés dans diverses applications.